# Functions demo
Just try to wrong all the functions available in the py files

In [1]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pathlib import Path

import numpy as np
import pandas as pd
import wrds

from settings import config

DATA_DIR = Path(config("DATA_DIR"))
WRDS_USERNAME = config("WRDS_USERNAME")
START_YEAR = config("START_YEAR")
END_YEAR = config("END_YEAR")

In [40]:
from pull_markit import load_markit_data
from pull_rf_data import load_fed_yield_curve, load_fred_data

In [41]:
markit = load_markit_data()
fed_data = load_fed_yield_curve()
fred_data = load_fred_data()

In [295]:
markit

,ticker,redcode,month,first_trade_date,midspread,Date
0,EMN,29EB75,2001-08-01 00:00:00-04:00,2001-08-10,0.011089,2001-08-10
1,TIN,8FGHEE,2001-05-01 00:00:00-04:00,2001-05-01,0.011838,2001-05-01
2,LNC,5I759F,2001-11-01 01:00:00-04:00,2001-11-01,0.004350,2001-11-01
3,JCP,UB78A0,2001-12-01 01:00:00-04:00,2001-12-03,0.041375,2001-12-03
4,BDK,09A8GG,2001-07-01 00:00:00-04:00,2001-07-02,0.006541,2001-07-02
...,...,...,...,...,...,...
7902,CONRES,2D3216,2024-05-01 00:00:00-04:00,2024-05-01,0.009583,2024-05-01
7903,DIAMSPO,2H79EM,2024-03-01 01:00:00-04:00,2024-03-01,NaN,2024-03-01
7904,ILNOIS,4J7365,2024-02-01 01:00:00-04:00,2024-02-01,0.008737,2024-02-01
7905,WMI,9D516R,2024-11-01 00:00:00-04:00,2024-11-01,0.005239,2024-11-01


In [284]:
from calc_cds_t_return import load_cds_return

cds_return = load_cds_return()

In [297]:
cds_return["RD"].describe()

count    219252.000000
mean          4.776166
std           0.278877
min           0.007263
25%           4.762423
50%           4.841254
75%           4.893120
max           4.990784
Name: RD, dtype: float64

In [285]:
cds_return[(cds_return["month"] == "2001-01-01") & (cds_return["return"].notna())]

,ticker,month,trade_date,prev_spread,spread,RD,return


In [228]:
def generate_month_code(date):
    year = date.year
    month = date.month

    if month > 10:
        return f"{year}{month}"
    else:
        return f"{year}0{month}"

   

In [274]:
date = pd.to_datetime(np.nan)
generate_month_code(date)

'nan0nan'

In [286]:
cds_return["month_code"] = cds_return["month"].apply(generate_month_code)

In [287]:
def construct_cds_portfolios(cds_df, num_portfolios=20):
    """
    Constructs CDS portfolios by sorting firms into equal-sized portfolios 
    each month based on their CDS spreads.

    Args:
        cds_df (pd.DataFrame): DataFrame with 'ticker', 'month', 'spread'.
        num_portfolios (int): Number of portfolios (default = 20).

    Returns:
        pd.DataFrame: Portfolio assignment for each firm by month.
    """
    # Rank firms by CDS spread each month
    cds_df["spread_rank"] = cds_df.groupby("month_code")["spread"].rank(method="first")

    # Assign firms into portfolios (1 = lowest spreads, num_portfolios = highest spreads)
    cds_df["portfolio"] = cds_df.groupby("month_code")["spread"].transform(
        lambda x: pd.qcut(x, num_portfolios, labels=False) + 1
    )

    return cds_df[["ticker", "month_code", "spread", "portfolio", "return"]]

In [289]:
portfolio = construct_cds_portfolios(cds_return)

In [290]:

def calc_avg_portfolio_return(portfolio_df):
    """
    Computes the average return for each CDS portfolio per month.

    Args:
        portfolio_df (pd.DataFrame): DataFrame with 'month', 'portfolio', 'return'.

    Returns:
        pd.DataFrame: Portfolio-level average returns per month.
    """
    # Compute equal-weighted average return per portfolio each month
    portfolio_avg_return = portfolio_df.groupby(["month_code", "portfolio"])["return"].mean().reset_index()

    return portfolio_avg_return

In [292]:
df = calc_avg_portfolio_return(portfolio)

In [294]:
df[df["month_code"] == "200102"]

,month_code,portfolio,return
20,200102,1.0,-0.000315
21,200102,2.0,-0.000026
22,200102,3.0,0.000207
23,200102,4.0,0.000567
24,200102,5.0,-0.001689
25,200102,6.0,0.000144
26,200102,7.0,0.000565
27,200102,8.0,-0.000139
28,200102,9.0,0.000986
29,200102,10.0,0.000476
